In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/sentiment-analysis-dataset/train.csv")
test_data = pd.read_csv("/kaggle/input/sentiment-analysis-dataset/test.csv")

In [ ]:
train_data.head()

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
%matplotlib inline

In [ ]:
train_data[train_data.columns].isnull().sum()

In [ ]:
n1 = train_data.nunique(axis=0)

In [ ]:
n1

In [ ]:
dataplot = sns.heatmap(train_data.corr(), cmap="YlGnBu", annot=True)
plt.show()

In [ ]:

print(train_data.corr())

In [ ]:
test_data[test_data.columns].isnull().sum()

In [ ]:
test_data.shape

In [ ]:
n2 = test_data.nunique(axis=0)

In [ ]:
n2

In [ ]:
train_data = train_data.dropna()

In [ ]:
train_data.shape

In [ ]:
train_data[train_data.columns].isnull().sum()

In [ ]:
test_data.shape

In [ ]:
test_data = test_data.dropna()

In [ ]:
test_data.shape

In [ ]:
test_data[test_data.columns].isnull().sum()

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['sentiment'] = le.fit_transform(train_data['sentiment'])
test_data['sentiment'] = le.fit_transform(test_data['sentiment'])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB # classifier 
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    plot_confusion_matrix,
)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# text preprocessing modules
from string import punctuation 
# text preprocessing modules
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import re

In [ ]:
stop_words =  stopwords.words('english')
def text_cleaning(text, remove_stop_words=True, lemmatize_words=True):
    # Clean the text, with the option to remove stop_words and to lemmatize word
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"\'s", " ", text)
    text =  re.sub(r'http\S+',' link ', text)
    text = re.sub(r'\b\d+(?:\.\d+)?\s+', '', text) # remove numbers
        
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if lemmatize_words:
        text = text.split()
        lemmatizer = WordNetLemmatizer() 
        lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
        text = " ".join(lemmatized_words)
    
    # Return a list of words
    return(text)

In [ ]:
#clean the review
train_data["text"] = train_data["text"].apply(text_cleaning)
test_data["text"] = test_data["text"].apply(text_cleaning)
train_data["selected_text"] = train_data["selected_text"].apply(text_cleaning)

In [ ]:
train_data.head()

In [ ]:
train_data.dtypes

In [ ]:
test_data.head()

# Sentiment Analysis on Text

In [ ]:
train_input = []
train_label = []
test_label = []
test_input = []
for i in range(0,27480):
    a = train_data.iloc[i:i+1,1:2]
    b = train_data.iloc[i:i+1,3:4]
    train_input.append(str(a))
    train_label.append(b)
    
    

In [ ]:
for j in range(0,3534):
    c = test_data.iloc[j:j+1,1:2]
    d = test_data.iloc[j:j+1,2:3]
    test_input.append(str(c))
    test_label.append(d)

In [ ]:
training_labels_final = np.array(train_label)
testing_labels_final = np.array(test_label)

In [ ]:
vocab_size = 10000
embedding_dim = 20
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(train_input)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_input)
padded = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

In [ ]:
testing_sequences = tokenizer.texts_to_sequences(test_input)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [ ]:
padded

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(train_input[1])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(48, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
]
)
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
num_epochs = 20
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

In [ ]:

import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

# Training on Selected Text

In [ ]:
train_input1 = []
train_label1 = []
for i in range(0,27480):
    a = train_data.iloc[i:i+1,1:2]
    b = train_data.iloc[i:i+1,3:4]
    train_input1.append(str(a))
    train_label1.append(b)

In [ ]:
training_labels_final1 = np.array(train_label1)

In [ ]:
vocab_size = 1200
embedding_dim = 40
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(train_input1)
word_index = tokenizer.word_index
sequences1 = tokenizer.texts_to_sequences(train_input1)
padded1 = pad_sequences(sequences1,maxlen=max_length,truncating=trunc_type)

In [ ]:
testing_sequences1 = tokenizer.texts_to_sequences(test_input)
testing_padded1 = pad_sequences(testing_sequences1,maxlen=max_length)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(48, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
]
)
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
num_epochs = 20
history = model.fit(padded1, training_labels_final1, epochs=num_epochs, validation_data=(testing_padded1, testing_labels_final))

In [ ]:
model.save('Sentiment_Analysis')

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, tokenizer.vocab_size):
  word = tokenizer.decode([word_num])
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()
